# Question Answering with Sources

This notebook walks through how to use LangChain for question answering with sources over a list of documents. It covers four different chain types: `stuff`, `map_reduce`, `refine`,`map-rerank`. For a more in depth explanation of what these chain types are, see [here](../combine_docs.md).

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

API_KEY = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = API_KEY

## Prepare Data
First we prepare the data. For this example we do similarity search over a vector database, but these documents could be fetched in any manner (the point of this notebook to highlight what to do AFTER you fetch the documents).

In [10]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.vectorstores.faiss import FAISS
import pickle

In [18]:
from langchain.document_loaders import UnstructuredFileLoader

# load raw data
loader = UnstructuredFileLoader("./csci.pdf", mode="elements")
raw_documents = loader.load()

In [19]:
docs = raw_documents
print(len(docs))
for i in range(len(docs)):
    print(docs[i])
    # print(docs[i].metadata['category'], docs[i].page_content)
    
# Merge NarrativeText up to some max length
# Include previous title(s) before Narrative Text
# Prob just the single prev title, but maybe multiple? Maybe use overlap?
# Remove UncategorizedText?

261
page_content='Daniel W. Barowy, Assistant Professor of Computer Science  James M. Bern, Assistant Professor of Computer Science  Rohit Bhattacharya, Assistant Professor of Computer Science  Stephen N. Freund, Chair and A. Barton Hepburn Professor of Computer Science  Mark Hopkins, Assistant Professor of Computer Science  Iris Howley, Assistant Professor of Computer Science  Bill K. Jannen, Assistant Professor of Computer Science  Katie A. Keith, Assistant Professor of Computer Science  Samuel M' metadata={'source': './csci.pdf', 'filename': './csci.pdf', 'page_number': 1, 'category': 'ListItem'}
page_content='auley, Assistant Professor of Computer Science; on leave' metadata={'source': './csci.pdf', 'filename': './csci.pdf', 'page_number': 1, 'category': 'ListItem'}
page_content='-' metadata={'source': './csci.pdf', 'filename': './csci.pdf', 'page_number': 1, 'category': 'ListItem'}
page_content='Kelly A. Shaw, Professor of Computer Science  Shikha Singh, Assistant Professor of Com

In [13]:
def merge_narrative_text(raw_documents):
    for doc in documents:
        print(doc)

In [20]:
# merge_narrative_text(raw_documents)

In [21]:
# Split text
text_splitter = CharacterTextSplitter(chunk_size = 2000, chunk_overlap  = 200)
documents = text_splitter.split_documents(raw_documents)

In [22]:
documents

[Document(page_content='Daniel W. Barowy, Assistant Professor of Computer Science  James M. Bern, Assistant Professor of Computer Science  Rohit Bhattacharya, Assistant Professor of Computer Science  Stephen N. Freund, Chair and A. Barton Hepburn Professor of Computer Science  Mark Hopkins, Assistant Professor of Computer Science  Iris Howley, Assistant Professor of Computer Science  Bill K. Jannen, Assistant Professor of Computer Science  Katie A. Keith, Assistant Professor of Computer Science  Samuel M', metadata={'source': './csci.pdf', 'filename': './csci.pdf', 'page_number': 1, 'category': 'ListItem'}),
 Document(page_content='auley, Assistant Professor of Computer Science; on leave', metadata={'source': './csci.pdf', 'filename': './csci.pdf', 'page_number': 1, 'category': 'ListItem'}),
 Document(page_content='-', metadata={'source': './csci.pdf', 'filename': './csci.pdf', 'page_number': 1, 'category': 'ListItem'}),
 Document(page_content='Kelly A. Shaw, Professor of Computer Scie

In [29]:
# Load Data to vectorstore
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(documents, embeddings)


# Save vectorstore
with open("./vectorstore.pkl", "wb") as f:
    pickle.dump(vectorstore, f)

# embeddings = OpenAIEmbeddings()
# db = Chroma.from_documents(documents, embeddings)

In [30]:
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI

qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), vectorstore.as_retriever(), return_source_documents=True)

In [31]:
chat_history = []
query = "What is the full name of the course 'Computer Science 256?'"
result = qa({"question": query, "chat_history": chat_history})

In [32]:
result["answer"]

' Computer Science 256 is the full name of the course.'

In [35]:
result['source_documents']

[Document(page_content='Computer Science 256 Algorithm Design and Analysis', metadata={'source': './csci.pdf', 'filename': './csci.pdf', 'page_number': 2, 'category': 'Title'}),
 Document(page_content='Prerequisites:  CSCI 256', metadata={'source': './csci.pdf', 'filename': './csci.pdf', 'page_number': 12, 'category': 'Title'}),
 Document(page_content='general programming, Computer Science 134 followed by 136 and 256 will provide students with a strong background in algorithm and data structure', metadata={'source': './csci.pdf', 'filename': './csci.pdf', 'page_number': 4, 'category': 'NarrativeText'}),
 Document(page_content='Computer Science 134 Introduction to Computer Science', metadata={'source': './csci.pdf', 'filename': './csci.pdf', 'page_number': 1, 'category': 'Title'})]

In [33]:
query = "What is the Discrete Mathematics Proficiency Requirement?"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

' The Discrete Mathematics Proficiency Requirement is the requirement that students must demonstrate proficiency in discrete mathematics by either passing the departmental Discrete Mathematics Proficiency Exam or by earning a grade of C- or better in MATH 200.'

In [34]:
query = "How many CS courses do I need to take to graduate?"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

' You need to take a minimum of 8 courses in Computer Science, including the four core courses (Computer Science 237, 256, 334, and 361) and two or more electives (bringing the total number of Computer Science courses to at least 8).'

In [35]:
query = "I haven't taken CS237. Can I take Computer Graphics?"
result = qa({"question": query, "chat_history": chat_history})
chat_history = [(query, result["answer"])]
result["answer"]

' No, you cannot take Computer Graphics since it requires CSCI 256 and CSCI 237 as prerequisites.'

In [50]:
query = "Why not?"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

' CSCI 237 is a prerequisite for CSCI 371 Computer Graphics because it provides the foundational knowledge needed to understand the concepts taught in the course.'

In [8]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI

## Quickstart
If you just want to get started as quickly as possible, this is the recommended way to do it:

In [9]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")
query = "What did the president say about Justice Breyer"
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': ' The president thanked Justice Breyer for his service.\nSOURCES: 30-pl'}

If you want more control and understanding over what is happening, please see the information below.

## The `stuff` Chain

This sections shows results of using the `stuff` Chain to do question answering with sources.

In [6]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")

In [7]:
query = "What did the president say about Justice Breyer"
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': ' The president thanked Justice Breyer for his service.\nSOURCES: 30-pl'}

**Custom Prompts**

You can also use your own prompts with this chain. In this example, we will respond in Italian.

In [7]:
template = """Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer.
Respond in Italian.

QUESTION: {question}
=========
{summaries}
=========
FINAL ANSWER IN ITALIAN:"""
PROMPT = PromptTemplate(template=template, input_variables=["summaries", "question"])

chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff", prompt=PROMPT)
query = "What did the president say about Justice Breyer"
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': '\nNon so cosa abbia detto il presidente riguardo a Justice Breyer.\nSOURCES: 30, 31, 33'}

## The `map_reduce` Chain

This sections shows results of using the `map_reduce` Chain to do question answering with sources.

In [8]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="map_reduce")

In [9]:
query = "What did the president say about Justice Breyer"
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': ' The president thanked Justice Breyer for his service.\nSOURCES: 30-pl'}

**Intermediate Steps**

We can also return the intermediate steps for `map_reduce` chains, should we want to inspect them. This is done with the `return_map_steps` variable.

In [16]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="map_reduce", return_intermediate_steps=True)

In [17]:
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'intermediate_steps': [' "Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service."',
  ' None',
  ' None',
  ' None'],
 'output_text': ' The president thanked Justice Breyer for his service.\nSOURCES: 30-pl'}

**Custom Prompts**

You can also use your own prompts with this chain. In this example, we will respond in Italian.

In [8]:

question_prompt_template = """Use the following portion of a long document to see if any of the text is relevant to answer the question. 
Return any relevant text in Italian.
{context}
Question: {question}
Relevant text, if any, in Italian:"""
QUESTION_PROMPT = PromptTemplate(
    template=question_prompt_template, input_variables=["context", "question"]
)

combine_prompt_template = """Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer.
Respond in Italian.

QUESTION: {question}
=========
{summaries}
=========
FINAL ANSWER IN ITALIAN:"""
COMBINE_PROMPT = PromptTemplate(
    template=combine_prompt_template, input_variables=["summaries", "question"]
)

chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="map_reduce", return_intermediate_steps=True, question_prompt=QUESTION_PROMPT, combine_prompt=COMBINE_PROMPT)
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'intermediate_steps': ["\nStasera vorrei onorare qualcuno che ha dedicato la sua vita a servire questo paese: il giustizia Stephen Breyer - un veterano dell'esercito, uno studioso costituzionale e un giustizia in uscita della Corte Suprema degli Stati Uniti. Giustizia Breyer, grazie per il tuo servizio.",
  ' Non pertinente.',
  ' Non rilevante.',
  " Non c'è testo pertinente."],
 'output_text': ' Non conosco la risposta. SOURCES: 30, 31, 33, 20.'}

**Batch Size**

When using the `map_reduce` chain, one thing to keep in mind is the batch size you are using during the map step. If this is too high, it could cause rate limiting errors. You can control this by setting the batch size on the LLM used. Note that this only applies for LLMs with this parameter. Below is an example of doing so:

```python
llm = OpenAI(batch_size=5, temperature=0)
```

## The `refine` Chain

This sections shows results of using the `refine` Chain to do question answering with sources.

In [12]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="refine")

In [13]:
query = "What did the president say about Justice Breyer"
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': "\n\nThe president said that he was honoring Justice Breyer for his dedication to serving the country and that he was a retiring Justice of the United States Supreme Court. He also thanked him for his service and praised his career as a top litigator in private practice, a former federal public defender, and a family of public school educators and police officers. He noted Justice Breyer's reputation as a consensus builder and the broad range of support he has received from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. He also highlighted the importance of securing the border and fixing the immigration system in order to advance liberty and justice, and mentioned the new technology, joint patrols, dedicated immigration judges, and commitments to support partners in South and Central America that have been put in place. He also expressed his commitment to the LGBTQ+ community, noting the need for the bipartisan Equality Act and th

**Intermediate Steps**

We can also return the intermediate steps for `refine` chains, should we want to inspect them. This is done with the `return_intermediate_steps` variable.

In [18]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="refine", return_intermediate_steps=True)

In [19]:
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'intermediate_steps': ['\nThe president said that he was honoring Justice Breyer for his dedication to serving the country and that he was a retiring Justice of the United States Supreme Court. He also thanked Justice Breyer for his service.',
  '\n\nThe president said that he was honoring Justice Breyer for his dedication to serving the country and that he was a retiring Justice of the United States Supreme Court. He also thanked Justice Breyer for his service, noting his background as a top litigator in private practice, a former federal public defender, and a family of public school educators and police officers. He praised Justice Breyer for being a consensus builder and for receiving a broad range of support from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. He also noted that in order to advance liberty and justice, it was necessary to secure the border and fix the immigration system, and that the government was taking steps to do both. \

**Custom Prompts**

You can also use your own prompts with this chain. In this example, we will respond in Italian.

In [9]:
refine_template = (
    "The original question is as follows: {question}\n"
    "We have provided an existing answer, including sources: {existing_answer}\n"
    "We have the opportunity to refine the existing answer"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{context_str}\n"
    "------------\n"
    "Given the new context, refine the original answer to better "
    "answer the question (in Italian)"
    "If you do update it, please update the sources as well. "
    "If the context isn't useful, return the original answer."
)
refine_prompt = PromptTemplate(
    input_variables=["question", "existing_answer", "context_str"],
    template=refine_template,
)


question_template = (
    "Context information is below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question in Italian: {question}\n"
)
question_prompt = PromptTemplate(
    input_variables=["context_str", "question"], template=question_template
)

In [10]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="refine", return_intermediate_steps=True, question_prompt=question_prompt, refine_prompt=refine_prompt)
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'intermediate_steps': ['\nIl presidente ha detto che Justice Breyer ha dedicato la sua vita al servizio di questo paese e ha onorato la sua carriera.',
  "\n\nIl presidente ha detto che Justice Breyer ha dedicato la sua vita al servizio di questo paese, ha onorato la sua carriera e ha contribuito a costruire un consenso. Ha ricevuto un ampio sostegno, dall'Ordine Fraterno della Polizia a ex giudici nominati da democratici e repubblicani. Inoltre, ha sottolineato l'importanza di avanzare la libertà e la giustizia attraverso la sicurezza delle frontiere e la risoluzione del sistema di immigrazione. Ha anche menzionato le nuove tecnologie come scanner all'avanguardia per rilevare meglio il traffico di droga, le pattuglie congiunte con Messico e Guatemala per catturare più trafficanti di esseri umani, l'istituzione di giudici di immigrazione dedicati per far sì che le famiglie che fuggono da per",
  "\n\nIl presidente ha detto che Justice Breyer ha dedicato la sua vita al servizio di ques

## The `map-rerank` Chain

This sections shows results of using the `map-rerank` Chain to do question answering with sources.

In [10]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="map_rerank", metadata_keys=['source'], return_intermediate_steps=True)

In [11]:
query = "What did the president say about Justice Breyer"
result = chain({"input_documents": docs, "question": query}, return_only_outputs=True)

In [12]:
result["output_text"]

' The President thanked Justice Breyer for his service and honored him for dedicating his life to serve the country.'

In [14]:
result["intermediate_steps"]

[{'answer': ' The President thanked Justice Breyer for his service and honored him for dedicating his life to serve the country.',
  'score': '100'},
 {'answer': ' This document does not answer the question', 'score': '0'},
 {'answer': ' This document does not answer the question', 'score': '0'},
 {'answer': ' This document does not answer the question', 'score': '0'}]

**Custom Prompts**

You can also use your own prompts with this chain. In this example, we will respond in Italian.

In [11]:
from langchain.output_parsers import RegexParser

output_parser = RegexParser(
    regex=r"(.*?)\nScore: (.*)",
    output_keys=["answer", "score"],
)

prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

In addition to giving an answer, also return a score of how fully it answered the user's question. This should be in the following format:

Question: [question here]
Helpful Answer In Italian: [answer here]
Score: [score between 0 and 100]

Begin!

Context:
---------
{context}
---------
Question: {question}
Helpful Answer In Italian:"""
PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"],
    output_parser=output_parser,
)
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="map_rerank", metadata_keys=['source'], return_intermediate_steps=True, prompt=PROMPT)
query = "What did the president say about Justice Breyer"
result = chain({"input_documents": docs, "question": query}, return_only_outputs=True)

In [12]:
result

{'source': 30,
 'intermediate_steps': [{'answer': ' Il presidente ha detto che Justice Breyer ha dedicato la sua vita a servire questo paese e ha onorato la sua carriera.',
   'score': '100'},
  {'answer': ' Il presidente non ha detto nulla sulla Giustizia Breyer.',
   'score': '100'},
  {'answer': ' Non so.', 'score': '0'},
  {'answer': ' Il presidente non ha detto nulla sulla giustizia Breyer.',
   'score': '100'}],
 'output_text': ' Il presidente ha detto che Justice Breyer ha dedicato la sua vita a servire questo paese e ha onorato la sua carriera.'}